# Making Dashboard with Streamlit

In [ ]:
# Import libraries

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime as dt
from streamlit_keplergl import keplergl_static

In [ ]:
df = pd.read_csv('nyc-citibike_data.csv', index_col = 0, low_memory=False)
df.dtypes

In [ ]:
df.head()

In [ ]:
df_group = pd.read_csv('df_final_locations_for_map.csv')
df_group.dtypes

### Create a season column

In [ ]:
df['year_month'] = pd.to_datetime(df['year_month'], format='%Y-%m')

#Extract the month
df['month'] = df['year_month'].dt.month

df['season'] = [
"winter" if (month == 12 or 1 <= month <= 4)
    else "spring" if (4 < month <= 5)
    else "summer" if (6 <= month <= 9)
    else "fall"
for month in df['month']
    ]

In [ ]:
df[['year_month', 'month', 'season']].head()
df['season'].value_counts()

In [ ]:
df.shape

In [ ]:
df.columns

# plotly charts

In [ ]:
# Groupby
df['value'] = 1 
df_groupby_bar = df.groupby('start_station_name', as_index=False).agg({'value': 'sum'})
top20 = df_groupby_bar.nlargest(20, 'value')

In [ ]:
fig = go.Figure(go.Bar(x = top20['start_station_name'], y = top20['value'], marker={'color': top20['value'],'colorscale': 'Blues'}))
fig.show()

In [ ]:
## Bar chart

fig.update_layout(
    title = 'Top 20 most popular bike stations in New York',
    xaxis_title = 'Start stations',
    yaxis_title ='Sum of trips',
    width = 900, height = 600
)

In [ ]:
# Save the top 20 stations as a csv file 
top20.to_csv('top20_station.csv')

## Line charts

In [ ]:
# Create trip counts
trip_counts = df.groupby('date').size().reset_index(name='trip_count')

# Merge trip counts back to df
df_weather = df[['date', 'avgTemp']].drop_duplicates()
df_weather = pd.merge(df_weather, trip_counts, on='date', how='left').set_index('date')

In [ ]:
df_weather.to_csv('trips_weather.csv')

In [ ]:
df_weather.dtypes

In [ ]:
# Load and prepare data
df_weather = pd.read_csv('trips_weather.csv', parse_dates=['date'])
df_weather.sort_values('date', inplace=True)

# Create the dual-axis figure
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bike ride count line (left y-axis)
fig.add_trace(
    go.Scatter(
        x=df_weather['date'],
        y=df_weather['trip_count'],
        name='Daily Bike Rides',
        line=dict(color='blue')
    ),
    secondary_y=False
)

# Add temperature line (right y-axis)
fig.add_trace(
    go.Scatter(
        x=df_weather['date'],
        y=df_weather['avgTemp'],
        name='Avg Temperature (°C)',
        line=dict(color='red')
    ),
    secondary_y=True
)

# Customize layout
fig.update_layout(
    title='Bike Rides and Temperature Over Time in NYC',
    xaxis_title='Date',
    yaxis_title='Trip Count',
    yaxis2_title='Avg Temperature (°C)',
    legend=dict(x=0.01, y=0.99),
    plot_bgcolor='white',
    title_x=0.5
)

fig.show()

In [ ]:
# cleaning up 
import gc # this is a garbage collector
gc.collect()

In [ ]:
df.columns

In [ ]:
df_1 = df.drop(columns = {'ride_id', 
                          'rideable_type', 
                          'started_at', 
                          'ended_at', 
                          'start_station_id', 
                          'end_station_id', 
                          'member_casual', 
                          'year_month', 
                          '_merge'}
              ) 

In [ ]:
df_1.columns